In [1]:
import h5py
import pandas as pd
import numpy as np
import os
import random

In [4]:
def create_velocityfield_v_path(
    dir_data='./data_p/',
    num_particle=2
):
        #########################################
    """
    Use lag-{time} for coordinates and lag_velocityField 
    for plotting the velocityf field arrow.
    
    In paraview use: 
     iHat * vx + jHat * vy + kHat * vz
    """
    #########################################
    # Check if the data folder exists
    if not os.path.exists(dir_data):
        raise FileNotFoundError(f"Folder '{dir_data}' not found.")
    
    # count num of timesteps
    prefix = f'tip_velocitygradient'
    files = os.listdir(dir_data)
    num_files = sum([1 for file in files if file.startswith(prefix)])
    print(num_files)

    pd.set_option('display.precision', 20)
    columns = ['x', 'y', 'z', 'vx', 'vy', 'vz']
    df = pd.DataFrame(columns=columns)
    
    # loop by number of files
    print('hi')
    time = 0
    while time < num_files:
        # velfield
        h5_file =  dir_data + f'tip_velocityfield-{time}.h5'
            
        with h5py.File(h5_file, 'r') as file:
            for key in file.keys():
                vel_field = file[key][num_particle]
            
        #coordinates
        h5_file_coord =  dir_data +  f'tip-{time}.h5'
        with h5py.File(h5_file_coord, 'r') as file:
            # create a variable name for storing strain rate tensor array
            for key in file.keys():
                coordinates = file[key][num_particle]
            
        
        # construct df
        x = coordinates[0]
        y = coordinates[1]
        z = coordinates[2]
        vx = vel_field[0]
        vy = vel_field[1]
        vz = vel_field[2]
        new_row = {'x': x, 'y': y, 'z': z, 'vx': vx, 'vy': vy, 'vz': vz}
        df = df.append(new_row, ignore_index=True)
        
        # SAVE FILE
        save_path = './csv_velfield_vector_path/'
        # Check if destination folder exist
        os.makedirs(save_path, exist_ok=True)
        df.to_csv(save_path + f'velocity_vector_{num_particle}_{time}.csv', index=False)

        time += 1
    

In [5]:
create_velocityfield_v_path()

17
hi


/tmp/ipykernel_1176/2438638614.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_1176/2438638614.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_1176/2438638614.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_1176/2438638614.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_1176/2438638614.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

### In Paraview:

    a) Open -> CSV Reader -> Apply
    
    b)Filtetr -> Alphabetical -> Table to Points
    
        x = x
        y = y 
        z = z
        
        Apply
        
    c) Filter -> Alphabetical -> Calculator:
        iHat * vx + jHat * vy + kHat * vz
        
        Apply
        
    d) Filter -> Alphabetical -> Glyph
    
    Scalable Factor 250
    
    Apply
        